Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

2025-06-19 10:38:02.772356: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-19 10:38:02.881681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-19 10:38:02.881709: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-06-19 10:38:03.579021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [6]:
def create_windows(data, past, future):
    X, y = [], []
    for i in range(len(data) - past - future + 1):
        X.append(data[i:i+past])
        y.append(data[i+past+future-1])
    return np.array(X), np.array(y)

In [7]:
past, future = (5, 2)
X_train, y_train = create_windows(train_data, past, future)
X_test, y_test = create_windows(test_data, past, future)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [ ]:
inputs = keras.layers.Input(shape=(X_train.shape[1], 1))
x = layers.GRU(32, return_sequences=True)(inputs)
x = layers.GRU(16)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

2025-06-19 10:49:39.334425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-19 10:49:39.334464: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-19 10:49:39.334487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-1cf422): /proc/driver/nvidia/version does not exist
2025-06-19 10:49:39.334746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 1)]            0         
                                                                 
 gru (GRU)                   (None, 5, 32)             3360      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 5,777
Trainable params: 5,777
Non-trainable params: 0
_________________________________________________________________


In [9]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 [==============================] - 3s 17ms/step - loss: 97.5175 - val_loss: 80.4796
Epoch 2/200
38/38 [==============================] - 0s 4ms/step - loss: 50.4376 - val_loss: 53.4366
Epoch 3/200
38/38 [==============================] - 0s 4ms/step - loss: 36.7999 - val_loss: 43.7483
Epoch 4/200
38/38 [==============================] - 0s 4ms/step - loss: 31.1561 - val_loss: 37.7762
Epoch 5/200
38/38 [==============================] - 0s 4ms/step - loss: 27.2894 - val_loss: 33.1836
Epoch 6/200
38/38 [==============================] - 0s 4ms/step - loss: 24.3873 - val_loss: 29.5275
Epoch 7/200
38/38 [==============================] - 0s 4ms/step - loss: 22.2041 - val_loss: 26.6133
Epoch 8/200
38/38 [==============================] - 0s 4ms/step - loss: 20.5962 - val_loss: 24.2838
Epoch 9/200
38/38 [==============================] - 0s 4ms/step - loss: 19.4158 - val_loss: 22.5549
Epoch 10/200
38/38 [==============================] - 0s 4ms/step - loss: 18.5887 - val_lo

In [10]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 [==============================] - 0s 1ms/step - loss: 6.9346
Test Loss: 6.934563636779785


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [11]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [12]:
## Create windows
def create_multi_windows(data, past, future):
    X, y = [], []
    values = data.values
    for i in range(len(values) - past - future + 1):
        X.append(values[i:i+past])
        y.append(values[i+past+future-1][0])  # target is 'Temp'
    return np.array(X), np.array(y)

X_train, y_train = create_multi_windows(train_data, past, future)
X_test, y_test = create_multi_windows(test_data, past, future)

In [13]:
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = layers.GRU(32, return_sequences=True)(inputs)
x = layers.GRU(16)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 2)]            0         
                                                                 
 gru_2 (GRU)                 (None, 5, 32)             3456      
                                                                 
 gru_3 (GRU)                 (None, 16)                2400      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5,873
Trainable params: 5,873
Non-trainable params: 0
_________________________________________________________________


In [14]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 [==============================] - 3s 17ms/step - loss: 112.9391 - val_loss: 98.3845
Epoch 2/200
38/38 [==============================] - 0s 5ms/step - loss: 66.1965 - val_loss: 73.8562
Epoch 3/200
38/38 [==============================] - 0s 4ms/step - loss: 52.8861 - val_loss: 62.5856
Epoch 4/200
38/38 [==============================] - 0s 4ms/step - loss: 44.9291 - val_loss: 54.1231
Epoch 5/200
38/38 [==============================] - 0s 4ms/step - loss: 38.8159 - val_loss: 47.2922
Epoch 6/200
38/38 [==============================] - 0s 4ms/step - loss: 33.9767 - val_loss: 41.6160
Epoch 7/200
38/38 [==============================] - 0s 4ms/step - loss: 30.0743 - val_loss: 36.9636
Epoch 8/200
38/38 [==============================] - 0s 4ms/step - loss: 26.9771 - val_loss: 33.0636
Epoch 9/200
38/38 [==============================] - 0s 4ms/step - loss: 24.5106 - val_loss: 29.8811
Epoch 10/200
38/38 [==============================] - 0s 4ms/step - loss: 22.5641 - val_l

In [15]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 [==============================] - 0s 1ms/step - loss: 6.3135
Test Loss: 6.3134870529174805


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


Respuestas:
* **a)** Clasificación de sentimiento en textos.
* **b)** Verificación de voz para iniciar el ordenador.
* **d)** Un clasificador que clasifique piezas de música según su autor.

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


Respuesta: 
* **a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.
* **b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.
* **c)** Son una manera de realizar transfer learning en nlp.
* **d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.